# Sample workflow of `nhgisxwalk`

## Subset to Wyoming

For further background information see:
* **Schroeder, J. P**. 2007. *Target-density weighting interpolation and uncertainty evaluation for temporal analysis of census data*. Geographical Analysis 39 (3):311–335.

#### NHGIS [block crosswalks](https://www.nhgis.org/user-resources/geographic-crosswalks)

In [1]:
%load_ext watermark
%watermark

2020-05-05T23:29:14-04:00

CPython 3.7.6
IPython 7.13.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [2]:
import nhgisxwalk
import glob
import inspect
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
pandas     1.0.3
nhgisxwalk 0.0.1
numpy      1.18.1



### Source and target years for the crosswalk

In [3]:
source_year, target_year = "1990", "2010"

### Source-target building base

In [4]:
crosswalk_dir = "../../crosswalks"
base_xwalk_name = "/nhgis_blk%s_blk%s_gj.csv.zip" % (source_year, target_year)
base_xwalk_file = crosswalk_dir + base_xwalk_name
data_types = nhgisxwalk.str_types(["GJOIN%s"%source_year, "GJOIN%s"%target_year])
base_xwalk = pandas.read_csv(base_xwalk_file, index_col=0, dtype=data_types)
base_xwalk.head()

/Users/jgaboardi/miniconda3/envs/nhgis_env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00
0,NaN,G01000300107014085,0.0,0.0
1,NaN,G01000300107014086,0.0,0.0
2,NaN,G01000300107014089,0.0,0.0
3,NaN,G01000300107014091,0.0,0.0
4,NaN,G01000300107014109,0.0,0.0


### Convenience code shorthand/lookup

In [5]:
print(inspect.getsource(nhgisxwalk.valid_geo_shorthand))

def valid_geo_shorthand(shorthand_name=True):
    """Shorthand lookups for census geographies."""
    lookup = {
        "blk": "block",
        "bgp": "block group part",
        "bkg": "block group",
        "trt": "tract",
        "cty": "county",
    }
    if not shorthand_name:
        lookup = {v: k for k, v in lookup.items()}
    return lookup



In [6]:
nhgisxwalk.valid_geo_shorthand(shorthand_name=False)

{'block': 'blk',
 'block group part': 'bgp',
 'block group': 'bkg',
 'tract': 'trt',
 'county': 'cty'}

### Instantiate an `nhgisxwalk.GeoCrossWalk` object — Wyoming
##### see [nhgisxwalk.GeoCrossWalk](https://github.com/jGaboardi/nhgisxwalk/blob/92b4fe55de0a9c53d0315dcda8ec121faaf20aef/nhgisxwalk/geocrosswalk.py#L19) for full details

In [7]:
nhgisxwalk.desc_code_blk1990

{'Persons': {'Persons': 'Universe',
  'NP1': 'Source code',
  'ET1': 'NHGIS code',
  'Total': 'ET1001'},
 'Race': {'Persons': 'Universe',
  'NP6': 'Source code',
  'EUY': 'NHGIS code',
  'Total': 'ET1001',
  'White': 'EUY001',
  'Black': 'EUY002',
  'American Indian, Eskimo, or Aleut': 'EUY003',
  'Asian or Pacific Islander': 'EUY004',
  'Other race': 'EUY005'},
 'Housing Units': {'Housing Units': 'Universe',
  'NH1': 'Source code',
  'ESA': 'NHGIS code',
  'Total': 'ET1001'},
 'Tenure': {'Occupied housing units': 'Universe',
  'NH3': 'Source code',
  'ES1': 'NHGIS code',
  'Owner occupied': 'ES1001',
  'Renter occupied': 'ES1002'}}

In [19]:
input_vars = [
    nhgisxwalk.desc_code_blk1990["Persons"]["Total"],
    #nhgisxwalk.desc_code_blk1990["Housing Units"]["Total"]
]
input_vars

['ET1001']

In [11]:
input_var_tags = ["pop"]#, "hu"]

In [12]:
bgp1990_to_trt2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="trt",
    base_source_table="../../tabular_data/USA_1990_block/1990_block.csv",
    input_var=input_vars,
    weight_var=input_var_tags,
    stfips="56"
)
bgp1990_to_trt2010.xwalk.tail(15)

,bgp1990,trt2010,wt_pop
1304,G560045092255999999513009999999999921,G5600450951300,1.000000
1305,G560045092255999999513009999999999922,G5600450951300,1.000000
1306,G560045092255999999513009999999999923,G5600450951300,1.000000
1307,G560045092255999999513009999999999924,G5600450951300,1.000000
1308,G560045092255999999513009999999999925,G5600450951100,0.107631
1309,G560045092255999999513009999999999925,G5600450951300,0.892369
1310,G560045093520791259512009999999999926,G5600450951100,1.000000
1311,G560045093520791259512009999999999927,G5600450951100,1.000000
1312,G560045093520999999512009999999999921,G5600450951100,1.000000
1313,G560045093520999999512009999999999922,G5600450951100,1.000000


### Write crosswalk to file

In [14]:
res_dir = "../../results/"
bgp1990_to_trt2010.xwalk_to_file(loc=res_dir)

### Read crosswalk from file

In [17]:
fname = res_dir + "bgp1990_to_trt2010_56"
_bgp1990_to_trt2010 = nhgisxwalk.GeoCrossWalk.xwalk_from_file(fname)
_bgp1990_to_trt2010.tail(15)

,Unnamed: 0,bgp1990,trt2010,wt_pop
1304,1304,G560045092255999999513009999999999921,G5600450951300,1.000000
1305,1305,G560045092255999999513009999999999922,G5600450951300,1.000000
1306,1306,G560045092255999999513009999999999923,G5600450951300,1.000000
1307,1307,G560045092255999999513009999999999924,G5600450951300,1.000000
1308,1308,G560045092255999999513009999999999925,G5600450951100,0.107631
1309,1309,G560045092255999999513009999999999925,G5600450951300,0.892369
1310,1310,G560045093520791259512009999999999926,G5600450951100,1.000000
1311,1311,G560045093520791259512009999999999927,G5600450951100,1.000000
1312,1312,G560045093520999999512009999999999921,G5600450951100,1.000000
1313,1313,G560045093520999999512009999999999922,G5600450951100,1.000000


-----------------------------------------------